# Open-Notebook (LLM) - Architecture Overview

## 1. Purpose & Scope
**Goal:** Provide a clear, high-level architecture overview of the Open-Notebook system so that any developer or stakeholder can understand how the major components fit together.
**Scope:** This document covers the primary runtime architecture: frontend, backend, worker processes, database, external AI providers, and storage, but excludes deployment infrastructure details (for example containers, cloud infra, CI/CD).

---

## 2. Architecture Approach
We use a container-style documentation approach (inspired by the C4 model) to show:
- system context (how external actors interact)
- internal containers and how they communicate
- dependencies on external services

---

## 3. System Context
```text
[ User (browser) ]
|
| uses web UI and uploads or interacts
v
+- Open-Notebook System (all modules) -+
|                                      |
+--------------------------------------+
^      ^       ^
|      |       |
|  depends on  |
|      |       |
[ Storage ] [ AI Providers ] [ File/Data Storage ]
```

**Actors / External Dependencies:**
- User (via browser) interacts with UI and uploads files.
- Storage volumes or file store holds uploaded raw files and persistent data (for example database files, file uploads).
- AI Providers (embedding or LLM inference services) support the system.

**Visual Context Diagram**
![Open-Notebook system context](context-diagram.png)
<small>Figure: Context-level view exported from context-diagram.png showing actors and first-level dependencies.</small>

---

## 4. Container Diagram (internal decomposition)
```plantuml
@startuml OpenNotebookContainers
title Open-Notebook (LLM) Container Diagram

actor User as "User (browser)"

cloud AI_Providers as "AI Providers\n(OpenAI/Anthropic/Ollama/...)"
storage File_Storage as "Storage Volumes\nuploads + data files"

rectangle OpenNotebook_System {
    [Frontend (Next.js UI)] as FE
    [API Backend (FastAPI)] as API
    [Worker (background processing)] as WK
    [Model Manager (model loading)] as MM
    [Database (SurrealDB)] as DB
}

User --> FE : uses UI / uploads / reads notebooks
FE --> API : proxied /api calls (CRUD, search, chat)
API --> DB : CRUD (sources, notes, models, embeddings, metadata)
API --> WK : submit background tasks (process_source, embed, etc.)
WK --> MM : request model loading / inference model
MM --> AI_Providers : send model instantiation / embedding / inference requests
WK --> AI_Providers : send embedding / inference requests
AI_Providers --> WK : return embeddings / inference results
WK --> DB : write embeddings, processed text, metadata
API --> DB : call DB functions (vector_search, text_search) for search/chat
DB --> File_Storage : persist uploaded files + DB data
FE --> File_Storage : serve uploaded files if needed

@enduml
```

**Containers and Responsibilities**
- Frontend (Next.js UI): user-facing web application that handles rendering, navigation, uploads, and proxies API calls.
- API Backend (FastAPI): handles HTTP API requests such as CRUD, search, chat, and orchestrates background tasks.
- Worker (background processing): asynchronous tasks for processing uploads, chunking text, embedding, and rebuilding embeddings.
- Model Manager (ModelMgr): selects, loads, and manages embedding or LLM models and interacts with external AI providers.
- Database (SurrealDB): stores persistent data (metadata, embeddings, model info) and runs vector or text searches.
- External AI Providers: third-party embedding or LLM services (OpenAI, Anthropic, Ollama, etc.).
- Storage Volumes or File Storage: keeps raw uploads and durable database or user data.

## 5. Data and Process Flows
1. User uploads a source or file -> Frontend handles upload -> Backend API stores metadata in DB and saves file to storage.
2. Backend triggers Worker for ingestion -> Worker parses and chunks -> Worker requests embeddings via Model Manager -> embeddings return -> Worker writes embeddings and processed content to DB.
3. User search/chat request -> Frontend -> Backend API -> DB vector_search or text_search -> results return to Frontend.
4. Model Manager reads default model metadata from DB, instantiates embedding or LLM models using AI providers.

## 6. Why This Architecture
- Clear separation of concerns (UI, API, background processing, storage, model management) improves maintainability and scalability.
- Asynchronous worker keeps API responsive while heavy embedding or processing runs in the background.
- External AI providers keep the system modular and allow swapping providers as needed.
- Persistent database plus file storage protects user data, embeddings, and metadata.

## 7. How to Use or Maintain This Document
- Keep this document alongside the codebase (for example docs/architecture.md).
- Update the diagram and explanation whenever system boundaries or flows change.
- Store PlantUML source in version control so anyone can regenerate visual assets.
- Reference this doc during onboarding or reviews to understand system interactions.


![Open-Notebook system architecture](architecture.png)
<small>Figure: High-level diagram exported from architecture.png showing the full Open-Notebook data and control flows.</small>